In [1]:
import os
import sys
import talib as ta
syspath = os.path.split(os.path.abspath("."))[0]
sys.path.append(syspath)
import univ3api.simulation as sim
import univ3api.utils as utils
import importlib
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# run if simulation file is updated
importlib.reload(sim)

HOST = "mongodb://172.16.11.81:27017/ETHDATA"
source = sim.DataUtilMongoDB(HOST)

from datetime import datetime
import time
d1 = datetime(2021,5,10,9)
d2 = datetime(2021,11,10,9)
t1 = int(time.mktime(d1.timetuple()))
t2 = int(time.mktime(d2.timetuple()))
# print(d1.timetuple())
# print(time.mktime(d1.timetuple()))
# print(int(time.mktime(d1.timetuple())))
# swapdata = source.swap_by_time("SWAP_WETH_USDT_3000", t1, t2)
swapdata = source.swap_by_time("SWAP_WBTC_WETH_3000", t1, t2)
swapdata.tail()
pricedata = swapdata[["timestamp", "sqrtPriceX96"]]
# pricedata["price"] = pricedata["sqrtPriceX96"].apply(utils.PriceConverter(18, 6).x96_to_price)
pricedata["price"] = pricedata["sqrtPriceX96"].apply(utils.PriceConverter(8, 18).x96_to_price)

In [2]:
pricedata['datetime'] = pd.to_datetime(pricedata['timestamp'],unit='s')
priceDt = pricedata.set_index('datetime')
priceDt['date'] = list(map(lambda d: d.date(), priceDt.index))
priceDt

,timestamp,sqrtPriceX96,price,date
datetime,,,,
2021-05-10 01:00:57,1620608457,30685129890900272086470739815588235,15.000190,2021-05-10
2021-05-10 01:00:57,1620608457,30709533659969384051347723595723166,15.024059,2021-05-10
2021-05-10 01:03:12,1620608592,30705863068469425277565093729731105,15.020468,2021-05-10
2021-05-10 01:04:43,1620608683,30724919029063427727612188810824282,15.039117,2021-05-10
2021-05-10 01:05:15,1620608715,30722150531496348621299112840911173,15.036406,2021-05-10
...,...,...,...,...
2021-11-09 05:29:33,1636435773,29838382263891094853397020984797412,14.183760,2021-11-09
2021-11-09 05:29:58,1636435798,29830238196962177663795291495427951,14.176019,2021-11-09
2021-11-09 05:36:59,1636436219,29829209673960410593094947881286270,14.175041,2021-11-09


In [3]:
UDP60 = pd.read_csv('../../data/btceth/UDP60.csv',index_col=0)
MA = pd.read_csv('../../data/btceth/MA60.csv',index_col=0)
# MA60 = pd.read_csv('../data/symbolsSig/MA60.csv',index_col=0)
UDP60.dropna(axis = 0,how='any',inplace=True)
UDP60.index = pd.to_datetime(UDP60.index)
udpInd = UDP60.index.tolist()
MA.dropna(axis = 0,how='any',inplace=True)
MA.index = pd.to_datetime(MA.index)
maInd = MA.index.tolist()
# print(type(udpInd[0]))
# str(udpInd[0])[:-6]
strudpInd = [str(ind)[:-6] for ind in udpInd]
strmaInd = [str(ind)[:-6] for ind in maInd]
UDP60.index  = strudpInd
MA.index = strmaInd
# UDP60
MA.columns = ['btcCLOSEoverMA','ethCLOSEoverMA']

In [4]:
udpDict = dict(UDP60['udp60'])
maBTCDict = dict(MA['btcCLOSEoverMA'])
maETHDict = dict(MA['ethCLOSEoverMA'])
udpList = []
maBTCList = []
maETHList =  []
for i, v in priceDt.iterrows():
    k =str(pd.to_datetime(v['date']))
    udpList.append(udpDict[k])
    maBTCList.append(maBTCDict[k])
    maETHList.append(maETHDict[k])
print(f'len of udpList:{len(udpList)}')
priceDt['udp'] = udpList
print(f'len of maBTCList:{len(maBTCList)}')
print(f'len of maETHList:{len(maETHList)}')
priceDt['btcCLOSEoverMA'] = maBTCList
priceDt['ethCLOSEoverMA'] = maETHList

len of udpList:48857
len of maBTCList:48857
len of maETHList:48857


In [5]:
priceDt.drop(['date'],axis=1,inplace=True)
priceDt

,timestamp,sqrtPriceX96,price,udp,btcCLOSEoverMA,ethCLOSEoverMA
datetime,,,,,,
2021-05-10 01:00:57,1620608457,30685129890900272086470739815588235,15.000190,0.6,False,True
2021-05-10 01:00:57,1620608457,30709533659969384051347723595723166,15.024059,0.6,False,True
2021-05-10 01:03:12,1620608592,30705863068469425277565093729731105,15.020468,0.6,False,True
2021-05-10 01:04:43,1620608683,30724919029063427727612188810824282,15.039117,0.6,False,True
2021-05-10 01:05:15,1620608715,30722150531496348621299112840911173,15.036406,0.6,False,True
...,...,...,...,...,...,...
2021-11-09 05:29:33,1636435773,29838382263891094853397020984797412,14.183760,1.0,True,True
2021-11-09 05:29:58,1636435798,29830238196962177663795291495427951,14.176019,1.0,True,True
2021-11-09 05:36:59,1636436219,29829209673960410593094947881286270,14.175041,1.0,True,True


In [6]:
init_price = priceDt["price"][0]

decimal0 = 8 #BTC的精度
decimal1 = 18 #ETH的精度

init_amount0 = 1*1e8 #初始持有BTC的数量
init_amount1 = 0 #初始持有ETH的数量
print('init_price =',init_price)

init_price = 15.000190153257385


In [7]:
import btceth_udpMA_strategy
importlib.reload(utils)
importlib.reload(sim)
importlib.reload(btceth_udpMA_strategy)
strategy = btceth_udpMA_strategy.HoldStrategy(init_amount0, init_amount1, decimal0, decimal1, sim.PoolFee.low, True)
strategy.init(swapdata.iloc[0].to_dict())
# strategy.run(swapdata, priceDt)
strategy.run(swapdata, priceDt)
report = strategy.report()

**********************【创建long池子】【第1次建池】**********************
转换后钱包中余额 amount0:  29965000.0 amount1:  12154653216089452544
price:  17.36737020727188
$$$$$$$【Upper Price】: 26.05105531090782
$$$$$$$【Lower Price】: 8.68368510363594
######【L】:68230096213881【BTC】:29965000【ETH】:8333272167327284224
将要投入池子的数量 amount_t0: 29964999 amount_t1: 8333272167327288320
【RealWorldTime】:2021-07-29 07:05:27, Timestamp: 1627513527, Blocknumber: 12917197
Mint position： Position(L=68209224894886, tick=[251880, 262860], range=[0.0384, 0.1152])
【Mint amount】: token0=0.29964998, token1=8.329131451746147
Wallet amount: token0=2e-08, token1=3.8255217643433053
